In [1]:
import pandas as pd
import warnings

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

warnings.simplefilter("ignore")

# Пример парсинга данных

В примере приводится вариант парсинга вакансий по направлениям "Data analisis" и 'Data science' на сайте по поиску вакансий HeadHunter('https://hh.ru/').

**Цель задачи**: получение информации о колличетве вакансий по указанным направлениям с учетом грейда (junior, middle, senior, without grade), размещенным в конкретном населенном пункте или пунктах (в текущем варианте Россиия - Москва и Нижний Новгород(Нижегородская область))

В качестве используемого браузера выбран YandexBrowser.

In [2]:
options = webdriver.ChromeOptions()

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

binary_yandex_driver_file = Service(executable_path=r'C:\Users\lebeda\yandexdriver-23.9.0.2209-win64\yandexdriver.exe')

driver = webdriver.Chrome(service=binary_yandex_driver_file, options=options)

In [3]:
driver.get('https://hh.ru/')

In [4]:
#функция поиска элемента страницы по CSS-селектору
def find_element(page_element):
    return driver.find_element(By.CSS_SELECTOR, page_element)

In [5]:
#функция нажатия(клика) по найденному элементу
def click_element(page_element):
    return find_element(page_element).click()

In [6]:
#функция прокрутки(скроллинга) страницы до найденного элемента
def scroll_to_element(page_element):
    return driver.execute_script("arguments[0].scrollIntoView(true);", find_element(page_element))

In [7]:
#функция получения уточненного списка CSS-селекторов в случае получения данных по нескольким значениям элемента атрибутов поиска
#(например, выбор нескольких направлений специализаци и конкретных специальностей, выбор нескольких стран, регионов и городов)
def sharp_list(css_sel, sharp_elements_list):
    sharp_list = []
    for i in sharp_elements_list:
        sharp_element = css_sel.format(i)
        sharp_list.append(sharp_element)
    return sharp_list

In [8]:
#функция определения уровней(грейдов) списка полученных вакансий 
def grade_determ(vac):
    if ('junior' or 'jr') in vac:
        return 'junior'
    elif ('middle' or 'mid') in vac:
        return 'middle'
    elif ('senior' or 'sr') in vac:
        return 'senior'
    else:
        return 'without_grade'

**Получении CSS-селекторов для создания необходимой структуры поиска по сайту**

In [9]:
# кнопка cookie
cookies_button = 'button[class="bloko-button bloko-button_kind-primary"]'

In [10]:
#чек закрытия окна подтверждения выбора города (по имени класса)
city_confirm_window_cls_chk  = 'div[data-qa = "notification-close-button"]'

In [11]:
#кнопка "Расширенный поиск"
search_settings_btn = 'a[data-qa="advanced-search"]'

In [12]:
#кнопка "Указать специализацию"
exact_specialization_btn = 'button[data-qa = "resumesearch__profroles-switcher"]'

In [13]:
#определение списка искомых напарвлений специализации пункта "Специализация" 
css_sel_specialization_up_lev = 'span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-category-{}"][class="bloko-icon-link"]'
sharp_elements_specialization_up_lev = [11]
specialization_list_up_lev = sharp_list(css_sel_specialization_up_lev, sharp_elements_specialization_up_lev)
specialization_list_up_lev

['span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-category-11"][class="bloko-icon-link"]']

*Примечание:*

data-qa="data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-category-{}"" - атрибут выбора направления специализации тега span, где значения в {} - номер направления специализации согласно структуре источника, для примера:

11 - Информационные технологии

In [14]:
#определение списка искомых узких специализаций в разделах направлений пункта "Специализация"
css_sel_specialization_list_chek = 'span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-{}"][class="bloko-checkbox__text"]'
sharp_elements_specialization_list_chek = [156, 165]
specialization_list = sharp_list(css_sel_specialization_list_chek, sharp_elements_specialization_list_chek)
specialization_list

['span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-156"][class="bloko-checkbox__text"]',
 'span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-165"][class="bloko-checkbox__text"]']

*Примечание:*

data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-{}" - атрибут выбора специализации тега span, где значения в {} - номер специализации согласно структуре источника, для примера:

156 - Анализ данных

166 - Датасайнс

In [15]:
#кнопка "Выбрать" пункта "Специализация"
select_specialization_btn = 'button[data-qa="bloko-tree-selector-popup-submit"]'

In [16]:
#кнопка "Корзина" пункта "Регион"
search_region_del_but = 'div[data-qa="advanced-search__selected-regions"] > div > div > div > button'
#div2 > input[type='checkbox']

In [17]:
#кнопка "Список" пункта "Регион"
search_region_list_but= 'button[data-qa="advanced-search-region-selectFromList"]'

In [18]:
#чек-бокс "Россия" пункта "Регион"
search_region_rus_chek= 'span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-113"]'

In [19]:
#определение списка стран для поиска
css_sel_countries_list = 'span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-{}"]'
sharp_elements_countries_list = [113]
countries_list = sharp_list(css_sel_countries_list, sharp_elements_countries_list)
countries_list

['span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-113"]']

*Примечание:*

qata-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-{}" - атрибут выбора страны тега span, где значения в {} - номер страны согласно структуре источника:

9    - Айзербайджан

16   - Беларусь

28   - Грузия

40   - Казахстан

48   - Кыргызстан

113  - Россия

97   - Узбекистан

5    - Украина

1001 - другие регионы


In [20]:
css_sel_regions_list = 'span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-{}"]'
sharp_elements_regions_list = [1679]
regions_list = sharp_list(css_sel_regions_list, sharp_elements_regions_list)
regions_list

['span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-1679"]']

*Примечание:*

data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-{}" - атрибут выбора региона тега span, где значения в {} - номер региона согласно структуре источника, для примера:

1679 - Нижегородская область

In [21]:
css_sel_cities_list = 'span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-{}"]'
sharp_elements_cities_list = [1, 66]
cities_list = sharp_list(css_sel_cities_list, sharp_elements_cities_list)
cities_list

['span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-1"]',
 'span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-66"]']

*Примечание:*

data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-{}" - атрибут выбора населенного пункта тега span, где значения в {}- номер населенного пункта согласно структуре источника, для примера:

1  - Москва

66 - Нижний Новогород

In [22]:
#кнопка "Выбрать" пункта "Регион"
search_regions_cities_select_but = 'button[data-qa="bloko-tree-selector-popup-submit"]'

In [23]:
#кнопка "Найти"  "поиска вакансий"
search_btn = 'button[data-qa="advanced-search-submit-button"]'

In [24]:
#конечный список необходимых для навигации по сайту CSS-селекторов
css_selector_list = [city_confirm_window_cls_chk, search_settings_btn, exact_specialization_btn] + specialization_list_up_lev + specialization_list + [select_specialization_btn, search_region_del_but, search_region_list_but] + countries_list + regions_list + cities_list + [search_regions_cities_select_but, search_btn]
css_selector_list

['div[data-qa = "notification-close-button"]',
 'a[data-qa="advanced-search"]',
 'button[data-qa = "resumesearch__profroles-switcher"]',
 'span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-category-11"][class="bloko-icon-link"]',
 'span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-156"][class="bloko-checkbox__text"]',
 'span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-165"][class="bloko-checkbox__text"]',
 'button[data-qa="bloko-tree-selector-popup-submit"]',
 'div[data-qa="advanced-search__selected-regions"] > div > div > div > button',
 'button[data-qa="advanced-search-region-selectFromList"]',
 'span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-113"]',
 'span[data-qa="bloko-tree-selector-toogle-node bloko-tree-selector-toogle-node-1679"]',
 'span[data-qa="bloko-tree-selector-item-text bloko-tree-selector-item-text-1"]',
 'span[data-qa="bloko-tree-selector-item-text bloko-tree-sel

**Цикл навигации по сайту по заложенным параметрам поиска**

In [25]:
try:
    scroll_to_element(cookies_button)
    click_element(cookies_button)
    
finally:
    for i in css_selector_list:
        scroll_to_element(i)
        click_element(i)

**Блок обработки списка полученных вакансий и вывод результатов**

In [26]:
vacancies = driver.find_elements(By.CLASS_NAME, "serp-item__title")

In [27]:
vacancies_list = list(map(lambda x: x.text.lower(), vacancies))

In [28]:
driver.close()

In [29]:
print('\033[1m' + 'Перечень полученных вакансий:' + '\033[0m', '\n')
print(pd.Series(vacancies_list))

Перечень полученных вакансий: 

0                      разработчик qliksense (удаленно)
1     специалист по работе с данными (автобизнес, ав...
2                                 junior data scientist
3                                     data scientist ml
4                 аналитик данных junior (crm/campaign)
5                                        data scientist
6                                   junior data analyst
7            практикант (продуктовая аналитика (excel))
8                               data-science-специалист
9                                        инженер данных
10                                        дата-аналитик
11                         data analyst/аналитик данных
12                                             аналитик
13                         data scientist \ ml engineer
14                          аналитик данных sql (риски)
15                                          ml engineer
16                                      аналитик данных
17              

In [30]:
vac_grades = pd.Series(vacancies_list).apply(grade_determ).value_counts()

In [31]:
print('\033[1m' + 'Информация о грейдах в объеме полученных вакансий' + '\033[0m', '\n')
print(vac_grades)

Информация о грейдах в объеме полученных вакансий 

without_grade    44
junior            6
middle            1
dtype: int64
